In [71]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

# import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional, Flatten
# from tensorflow.keras.layers.embeddings import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import Sequential

In [72]:
#import contractions
import string
from tensorflow.keras.utils import to_categorical
import re

#from gensim.models import FastText

In [73]:
from rnn_data_saver import *

In [74]:
readedr = RnnDataSaver()
readedr.results_read_json()
#df = pd.read_csv('myfile.csv') 


In [129]:
first_song = readedr.songs_list[0]
second_song = readedr.songs_list[1]



song_array = readedr.songs_list

chords_total = 0

for song in song_array:
    chords_total = chords_total + len(song)

# 2 песни по 1500 пар (ак, псп лист), а мне нужно массив 3000 псп листов

X_pr_tr =  np.zeros((1, 24), dtype=float)
Y_pr_tr =  np.array([])

for song  in song_array:
    for i in range(len(song)):
        #X_pr_tr = X_pr_tr.append(np.array(song[i][1],dtype=float))
        arr = np.array(song[i][1], dtype=float)
        #print(arr)
        X_pr_tr = np.vstack((X_pr_tr, arr))
        Y_pr_tr = np.append(Y_pr_tr, song[i][0])
        
        
X_pr_tr = X_pr_tr[1:]
print(X_pr_tr.shape)

(18020, 24)


In [130]:
set_chords = set()
for lbl in Y_pr_tr:
    set_chords.add(lbl)

list_chords = list(set_chords)
ids_total = len(list_chords)
dict_ch_id = {list_chords[i]: i for i in range(0, len(list_chords))}
dict_id_ch = {i: list_chords[i] for i in range(0, len(list_chords))} 

Y_pr_tr = np.array([dict_ch_id.get(Y_pr_tr[i]) for i in range(0, len(Y_pr_tr))], dtype=int).reshape((chords_total,1))  

print(Y_pr_tr.shape)
    

(18020, 1)


In [131]:
wn = 300

cnt_wn = int(chords_total/wn)
#print("cnt_wn = " + str(cnt_wn))

X =  np.zeros((1, wn, 24), dtype=float)
Y =  np.zeros((1, wn, 1))


for i in range(0, cnt_wn):
    X = np.vstack((X, X_pr_tr[(i)*wn:(i+1)*wn].reshape((1, wn, 24))))
    Y = np.vstack((Y, Y_pr_tr[(i)*wn:(i+1)*wn].reshape((1, wn, 1))))
    #print((i+1)*wn)

trainX = X[1:]
trainY = Y[1:]
print(trainX.shape)
print(trainY.shape)

testX = trainX.copy()
testY = trainY.copy()

(60, 300, 24)
(60, 300, 1)


In [132]:
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [133]:
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

(60, 300, 24) (60, 300, 34) (60, 300, 24) (60, 300, 34)


In [134]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainY.shape[1]


In [135]:
print(n_timesteps, n_features, n_outputs)

300 24 300


In [146]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 20, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[2]
   # print("trainX {}, trainy {}, testX {}, texty {}".format(trainX.shape, trainy.shape, testX.shape, testy.shape))
    model = Sequential()
    model.add(Bidirectional(LSTM(200, input_shape=(n_timesteps,n_features), return_sequences=True)))
    model.add(Bidirectional(LSTM(100, input_shape=(n_timesteps,n_features), return_sequences=True)))
    #model.add(Bidirectional(LSTM(100, input_shape=(n_timesteps,n_features), return_sequences=True)))
    model.add(Dropout(0.4))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    print(model.summary())
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [147]:
def summarize_results(scores):
    print(scores)
    #m, s = mean(scores), std(scores)
    #print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
    print((scores[0]+scores[1]+scores[2])/3)

In [148]:
def run_experiment(repeats=3):
    # load data
    #trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainY, testX, testY)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [144]:
"""score = evaluate_model(trainX, trainY, testX, testY)
score = score * 100.0
print(score)"""

'score = evaluate_model(trainX, trainY, testX, testY)\nscore = score * 100.0\nprint(score)'

In [145]:
run_experiment()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_58 (Bidirectio (None, 300, 400)          360000    
_________________________________________________________________
bidirectional_59 (Bidirectio (None, 300, 200)          400800    
_________________________________________________________________
dropout_29 (Dropout)         (None, 300, 200)          0         
_________________________________________________________________
dense_58 (Dense)             (None, 300, 100)          20100     
_________________________________________________________________
dense_59 (Dense)             (None, 300, 34)           3434      
Total params: 784,334
Trainable params: 784,334
Non-trainable params: 0
_________________________________________________________________
None
>#1: 44.417
Model: "sequential_30"
_________________________________________________________________
Layer (